In [5]:
import pandas as pd
import math
from PIL import Image, ImageDraw

df1 = pd.read_csv('../Bildgenerierung/horizontale.csv')
df2 = pd.read_csv('../Bildgenerierung/strichmann.csv')
df3 = pd.read_csv('../Bildgenerierung/ostfalia.csv')
df4 = pd.read_csv('../Bildgenerierung/vertikale.csv')
df5 = pd.read_csv('../Bildgenerierung/tshirt.csv')
df6 = pd.read_csv('../Bildgenerierung/schlange.csv')
df7 = pd.read_csv('../Bildgenerierung/haus_vom_nikolaus.csv')
dfs = [df1, df2, df3, df4, df5, df6, df7]

def distancePointToLineBetweenPoints(P, A, B):
    AB = [B[0] - A[0], B[1] - A[1]]
    AP = [P[0] - A[0], P[1] - A[1]]

    length_AB = math.sqrt(AB[0] ** 2 + AB[1] ** 2)
    AB_unit = [AB[0] / length_AB, AB[1] / length_AB]

    dot_product = AP[0] * AB_unit[0] + AP[1] * AB_unit[1]
    projection = [AB_unit[0] * dot_product, AB_unit[1] * dot_product]

    I = [A[0] + projection[0], A[1] + projection[1]]
    AI = [A[0] - I[0], A[1] - I[1]]
    BI = [B[0] - I[0], B[1] - I[1]]
    length_AI = math.sqrt(AI[0] ** 2 + AI[1] ** 2)
    length_BI = math.sqrt(BI[0] ** 2 + BI[1] ** 2)
    #print(length_AB)
    #print(length_AI)
    #print(length_BI)
    if length_AB < length_AI + length_BI:
        BP = [P[0] - B[0], P[1] - B[1]]
        length_AP = math.sqrt(AP[0] ** 2 + AP[1] ** 2)
        length_BP = math.sqrt(BP[0] ** 2 + BP[1] ** 2)
        #print('aussen ' + str(min(length_AP, length_BP)) + str(P) + str(A) + str(B))
        return [min(length_AP, length_BP), True]
    
    #dot_product = AP[0] * AB[0] + AP[1] * AB[1]
    #projection_length = dot_product / length_AB
    #print('innen ' + str(math.sqrt(AP[0] ** 2 + AP[1] ** 2 - projection_length ** 2)) + str(P) + str(A) + str(B))
    PI = [P[0] - I[0], P[1] - I[1]]
    #return [math.sqrt(AP[0] ** 2 + AP[1] ** 2 - projection_length ** 2), False]
    return [math.sqrt(PI[0] ** 2 + PI[1] ** 2), False]
    
def calculateDistance(index, x, y):
  df = dfs[index]
  dist = [1000000, False]
  for index, row in df.iterrows():
    match row['type']:
      case 'l':
        distance = distancePointToLineBetweenPoints((x,y), (row['x1'], row['y1']), (row['x2'], row['y2']))
        if dist[0] > distance[0]:
          dist = distance
      case 'c':
        pass
        #print('Not implemented yet')
  return dist
  
#def a():
#    A = (500, 650)
#    B = (500, 300)
#    P = (560, 299)
#    print(distancePointToLineBetweenPoints(P, A, B))

#a()
point_size = 1
image1 = Image.open('../Bildgenerierung/horizontale.png')
image2 = Image.open('../Bildgenerierung/strichmann.png')
image3 = Image.open('../Bildgenerierung/ostfalia.png')
image4 = Image.open('../Bildgenerierung/vertikale.png')
image5 = Image.open('../Bildgenerierung/tshirt.png')
image6 = Image.open('../Bildgenerierung/schlange.png')
image7 = Image.open('../Bildgenerierung/haus_vom_nikolaus.png')

images = [image1, image2, image3, image4, image5, image6, image7]

draw = []
for image in images:
  draw.append(ImageDraw.Draw(image))

for n in range(1, 27):
  print('person nr. ' + str(n))
  df = pd.read_csv('../Versuchsdaten_vorverarbeitet/' + str(n) + '/real.csv')
  pictures = df.groupby('sublevel')
  imageIndex = 0

  for group_value, group_data in pictures:
    pic_dataframes = []
    pic_dataframes.append(group_data.copy())
    pic_df = pd.concat(pic_dataframes, ignore_index=True)
    
    for index, row in pic_df.iterrows():
      x = float(row['x'])
      y = float(row['y'])
      #draw[imageIndex].point((x, y), fill=(100, 100, 100))
      dist = calculateDistance(imageIndex, x, y)
      distance = dist[0]
      red = int(distance * 10 - 45)
      green = int(255 - distance * 10)
      blue = 0
      if red < 0:
        red = 0
      if green < 0:
        green = 0
      #if dist[1]:
      #  red = 0
      #  green = 0
      #  blue = 255
      draw[imageIndex].rectangle([x - point_size, y - point_size, x + point_size, y + point_size], fill=(red,green,blue))
    imageIndex += 1
imageIndex = 0
for image in images:
  image.save('../Visualisierungen/Real/image' + str(imageIndex) + '.png')
  imageIndex += 1
print('Done')

new_image = Image.new('RGB',(3* 1368, 2*912), (0,0,0))
new_image.paste(images[0],(0,0))
new_image.paste(images[1],(1368,0))
new_image.paste(images[2],(2736,0))
new_image.paste(images[3],(0,912))
new_image.paste(images[4],(1368,912))
new_image.paste(images[5],(2736,912))
new_image.save("../Visualisierungen/Real/image-merged-real.png")
new_image.show()


person nr. 1
person nr. 2
person nr. 3
person nr. 4
person nr. 5
person nr. 6
person nr. 7
person nr. 8
person nr. 9
person nr. 10
person nr. 11
person nr. 12
person nr. 13
person nr. 14
person nr. 15
person nr. 16
person nr. 17
person nr. 18
person nr. 19
person nr. 20
person nr. 21
person nr. 22
person nr. 23
person nr. 24
person nr. 25
person nr. 26
Done
